# 应用1 | ReAct Agent：让 LLM 使用工具

---

## 🎯 本节目标

构建一个**真正能使用工具的 AI Agent**：
- 使用真实 LLM (支持 OpenAI / Ollama / 本地模型)
- 实现完整的 ReAct (Reasoning + Acting) 循环
- 可扩展的工具系统

---

## 什么是 ReAct？

**ReAct = Reasoning + Acting**

Agent 的思考过程：
```
问题 → 思考(Thought) → 行动(Action) → 观察(Observation) → 思考 → ... → 答案
```

### 本例中的工具

| 工具 | 功能 | 真实实现 |
|:---|:---|:---|
| 🔢 Calculator | 数学计算 | 安全的 Python eval |
| 🔍 WebSearch | 网络搜索 | DuckDuckGo API |
| 📅 DateTime | 日期时间 | 系统时间 |
| 🌤️ Weather | 天气查询 | Open-Meteo API |

## 运行前：准备 Ollama 后端

如果你选择 Ollama 作为后端，请先完成准备步骤：
- 参见 [PREPARE_OLLAMA.ipynb](./PREPARE_OLLAMA.ipynb)

完成后再继续本 notebook。


---

## 0. 环境配置

In [ ]:
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import json
import re
import requests
from datetime import datetime
from typing import Dict, List, Callable, Any, Optional
from dataclasses import dataclass

# 导入统一的 LLM 后端
from utils.llm_backend import get_llm_backend, auto_detect_backend

print("✓ 环境准备完成!")

In [ ]:
# ============================================================
# 配置 LLM 后端 (选择一种)
# ============================================================

# 方式1: 自动检测 (推荐)
# llm = auto_detect_backend()

# 方式2: 使用 Ollama (本地免费，需先安装 Ollama)
# llm = get_llm_backend("ollama", model="qwen3:4b")

# 方式3: 使用 OpenAI (需要 API Key)
# llm = get_llm_backend("openai", model="gpt-3.5-turbo")

# 方式4: 使用本地 HuggingFace 模型
# llm = get_llm_backend("huggingface", model="Qwen/Qwen3-4B-Instruct")

def check_ollama_ready(host="http://localhost:11434"):
    try:
        import requests
        resp = requests.get(f"{host}/api/tags", timeout=2)
        return resp.status_code == 200
    except Exception:
        return False

# 默认使用 Ollama
if not check_ollama_ready():
    print("Ollama 未启动。请先执行: ollama serve")
    print("并下载模型: ollama pull qwen3:4b")
try:
    llm = get_llm_backend("ollama", model="qwen3:4b")
    # 测试连接
    test_response = llm.chat([{"role": "user", "content": "Say 'OK' if you can hear me."}])
    print(f"✓ LLM 连接成功: {test_response[:50]}...")
except Exception as e:
    print(f"⚠️ Ollama 不可用: {e}")
    print("尝试使用 OpenAI...")
    try:
        llm = get_llm_backend("openai", model="gpt-3.5-turbo")
        print("✓ 使用 OpenAI 后端")
    except:
        print("❌ 请配置 LLM 后端 (Ollama 或 OpenAI)")
        print("   Ollama: https://ollama.ai/download")
        print("   然后运行: ollama pull qwen3:4b")

---

## 1. 定义工具系统

每个工具包含：
- **名称**: 唯一标识符
- **描述**: LLM 用来决定何时使用
- **参数**: JSON Schema 格式
- **执行函数**: 实际的功能实现

In [ ]:
@dataclass
class Tool:
    """工具定义"""
    name: str
    description: str
    parameters: Dict[str, Any]
    func: Callable
    
    def run(self, **kwargs) -> str:
        """执行工具"""
        try:
            result = self.func(**kwargs)
            return str(result)
        except Exception as e:
            return f"Error: {type(e).__name__}: {str(e)}"
    
    def to_openai_format(self) -> Dict:
        """转换为 OpenAI Function Calling 格式"""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.parameters
            }
        }

In [ ]:
# ============================================================
# 工具实现
# ============================================================

# 🔢 计算器工具
def calculator(expression: str) -> str:
    """
    安全的数学计算器
    支持: +, -, *, /, **, sqrt, sin, cos, tan, log, pi, e
    """
    import math
    
    # 安全的命名空间
    safe_dict = {
        'sqrt': math.sqrt, 'sin': math.sin, 'cos': math.cos,
        'tan': math.tan, 'log': math.log, 'log10': math.log10,
        'pi': math.pi, 'e': math.e, 'abs': abs, 'round': round,
        'pow': pow, 'sum': sum, 'min': min, 'max': max,
    }
    
    # 只允许安全字符
    allowed_chars = set('0123456789+-*/.()[], sqrtincoalgepwbmxu')
    expr_clean = expression.replace(' ', '').lower()
    
    if not all(c in allowed_chars for c in expr_clean):
        return f"Error: Invalid characters in expression"
    
    try:
        result = eval(expression, {"__builtins__": {}}, safe_dict)
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {str(e)}"

calculator_tool = Tool(
    name="calculator",
    description="Perform mathematical calculations. Supports +, -, *, /, **, sqrt(), sin(), cos(), tan(), log(), pi, e.",
    parameters={
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "The mathematical expression to evaluate, e.g., '2 + 3 * 4' or 'sqrt(16)'"
            }
        },
        "required": ["expression"]
    },
    func=calculator
)

# 测试
print("Calculator 测试:")
print(f"  2 + 3 * 4 = {calculator('2 + 3 * 4')}")
print(f"  sqrt(16) = {calculator('sqrt(16)')}")
print(f"  sin(pi/2) = {calculator('sin(pi/2)')}")

In [ ]:
# 🔍 网络搜索工具 (使用 DuckDuckGo)
def web_search(query: str, max_results: int = 3) -> str:
    """
    使用 DuckDuckGo 进行网络搜索
    """
    try:
        # DuckDuckGo Instant Answer API
        url = "https://api.duckduckgo.com/"
        params = {
            "q": query,
            "format": "json",
            "no_html": 1,
            "skip_disambig": 1
        }
        
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        
        results = []
        
        # Abstract (主要答案)
        if data.get("Abstract"):
            results.append(f"Summary: {data['Abstract']}")
        
        # Related Topics
        for topic in data.get("RelatedTopics", [])[:max_results]:
            if isinstance(topic, dict) and "Text" in topic:
                results.append(f"- {topic['Text']}")
        
        if results:
            return "\n".join(results)
        else:
            return f"No results found for: {query}. Try a more specific query."
            
    except requests.exceptions.RequestException as e:
        return f"Search error: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"

search_tool = Tool(
    name="web_search",
    description="Search the web for information. Use this when you need current or factual information.",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query"
            }
        },
        "required": ["query"]
    },
    func=web_search
)

# 测试
print("\nWeb Search 测试:")
print(web_search("Python programming language"))

In [ ]:
# 📅 日期时间工具
def get_datetime(timezone: str = "local") -> str:
    """
    获取当前日期和时间
    """
    from datetime import datetime, timezone as tz
    
    now = datetime.now()
    
    return f"""Current Date and Time:
- Date: {now.strftime('%Y-%m-%d')}
- Time: {now.strftime('%H:%M:%S')}
- Day of Week: {now.strftime('%A')}
- Week Number: {now.isocalendar()[1]}
- Timestamp: {int(now.timestamp())}"""

datetime_tool = Tool(
    name="get_datetime",
    description="Get the current date and time information.",
    parameters={
        "type": "object",
        "properties": {
            "timezone": {
                "type": "string",
                "description": "Timezone (optional, default is local)",
                "default": "local"
            }
        },
        "required": []
    },
    func=get_datetime
)

# 测试
print("\nDateTime 测试:")
print(get_datetime())

In [ ]:
# 🌤️ 天气查询工具 (使用 Open-Meteo API - 免费无需 API Key)
def get_weather(city: str) -> str:
    """
    获取城市天气信息 (使用 Open-Meteo API)
    """
    try:
        # 首先获取城市坐标 (使用 Open-Meteo 的 geocoding)
        geo_url = "https://geocoding-api.open-meteo.com/v1/search"
        geo_response = requests.get(
            geo_url,
            params={"name": city, "count": 1, "language": "en"},
            timeout=10
        )
        geo_data = geo_response.json()
        
        if not geo_data.get("results"):
            return f"City not found: {city}"
        
        location = geo_data["results"][0]
        lat, lon = location["latitude"], location["longitude"]
        city_name = location.get("name", city)
        country = location.get("country", "")
        
        # 获取天气数据
        weather_url = "https://api.open-meteo.com/v1/forecast"
        weather_response = requests.get(
            weather_url,
            params={
                "latitude": lat,
                "longitude": lon,
                "current": "temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m",
                "timezone": "auto"
            },
            timeout=10
        )
        weather_data = weather_response.json()
        
        current = weather_data.get("current", {})
        
        # 天气代码映射
        weather_codes = {
            0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
            45: "Foggy", 48: "Depositing rime fog",
            51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
            61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
            71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow",
            95: "Thunderstorm", 96: "Thunderstorm with hail"
        }
        
        weather_code = current.get("weather_code", 0)
        weather_desc = weather_codes.get(weather_code, "Unknown")
        
        return f"""Weather in {city_name}, {country}:
- Conditions: {weather_desc}
- Temperature: {current.get('temperature_2m', 'N/A')}°C
- Humidity: {current.get('relative_humidity_2m', 'N/A')}%
- Wind Speed: {current.get('wind_speed_10m', 'N/A')} km/h"""
        
    except Exception as e:
        return f"Weather API error: {str(e)}"

weather_tool = Tool(
    name="get_weather",
    description="Get current weather information for a city.",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name, e.g., 'Tokyo', 'New York', 'London'"
            }
        },
        "required": ["city"]
    },
    func=get_weather
)

# 测试
print("\nWeather 测试:")
print(get_weather("Tokyo"))

In [ ]:
# 工具集合
ALL_TOOLS = [calculator_tool, search_tool, datetime_tool, weather_tool]

print("\n✓ 已注册工具:")
for tool in ALL_TOOLS:
    print(f"  - {tool.name}: {tool.description[:50]}...")

---

## 2. 构建 ReAct Agent

核心流程：
1. **接收问题** → 构建包含工具描述的 prompt
2. **LLM 思考** → 输出 Thought + Action
3. **执行工具** → 获取 Observation
4. **循环** → 直到 LLM 给出 Final Answer

In [ ]:
class ReActAgent:
    """
    ReAct Agent - 使用真实 LLM 进行推理和工具调用
    """
    
    SYSTEM_PROMPT = """You are a helpful AI assistant that can use tools to answer questions.

Available Tools:
{tool_descriptions}

To use a tool, respond in this EXACT format:
Thought: [Your reasoning about what to do]
Action: [tool_name]
Action Input: [input for the tool as JSON, e.g., {{"expression": "2+2"}} or {{"query": "search term"}}]

When you have enough information to answer, respond:
Thought: [Your final reasoning]
Final Answer: [Your complete answer to the user]

Important:
- Always think step by step
- Use tools when needed, don't guess
- Action Input must be valid JSON
- Give a Final Answer when you're done
"""
    
    def __init__(self, llm, tools: List[Tool], max_steps: int = 10):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
        self.max_steps = max_steps
        self.history = []
    
    def _get_tool_descriptions(self) -> str:
        """生成工具描述"""
        descriptions = []
        for name, tool in self.tools.items():
            params = tool.parameters.get("properties", {})
            param_str = ", ".join([f"{k}: {v.get('description', '')}" for k, v in params.items()])
            descriptions.append(f"- {name}: {tool.description}\n  Parameters: {param_str}")
        return "\n".join(descriptions)
    
    def _parse_response(self, response: str) -> Dict:
        """
        解析 LLM 响应
        
        Returns:
            dict with keys: thought, action, action_input, final_answer
        """
        result = {
            "thought": None,
            "action": None,
            "action_input": None,
            "final_answer": None
        }
        
        # 提取 Thought
        thought_match = re.search(r'Thought:\s*(.+?)(?=Action:|Final Answer:|$)', response, re.DOTALL)
        if thought_match:
            result["thought"] = thought_match.group(1).strip()
        
        # 提取 Final Answer
        final_match = re.search(r'Final Answer:\s*(.+?)$', response, re.DOTALL)
        if final_match:
            result["final_answer"] = final_match.group(1).strip()
            return result
        
        # 提取 Action
        action_match = re.search(r'Action:\s*([\w_]+)', response)
        if action_match:
            result["action"] = action_match.group(1).strip()
        
        # 提取 Action Input
        input_match = re.search(r'Action Input:\s*(.+?)(?=Thought:|Action:|Final Answer:|$)', response, re.DOTALL)
        if input_match:
            input_str = input_match.group(1).strip()
            try:
                # 尝试解析 JSON
                result["action_input"] = json.loads(input_str)
            except json.JSONDecodeError:
                # 如果不是 JSON，尝试简单解析
                result["action_input"] = {"input": input_str}
        
        return result
    
    def run(self, question: str, verbose: bool = True) -> str:
        """
        运行 Agent 回答问题
        
        Args:
            question: 用户问题
            verbose: 是否打印详细过程
            
        Returns:
            str: 最终答案
        """
        if verbose:
            print(f"\n{'='*60}")
            print(f"🤔 Question: {question}")
            print(f"{'='*60}")
        
        # 构建系统提示
        system_prompt = self.SYSTEM_PROMPT.format(
            tool_descriptions=self._get_tool_descriptions()
        )
        
        # 对话历史
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
        
        self.history = []
        
        for step in range(self.max_steps):
            if verbose:
                print(f"\n--- Step {step + 1} ---")
            
            # 调用 LLM
            try:
                response = self.llm.chat(messages, temperature=0.1)
            except Exception as e:
                if verbose:
                    print(f"❌ LLM Error: {e}")
                return f"Error: {e}"
            
            if verbose:
                print(f"🤖 LLM Response:\n{response}")
            
            # 解析响应
            parsed = self._parse_response(response)
            
            # 记录历史
            self.history.append({
                "step": step + 1,
                "thought": parsed["thought"],
                "action": parsed["action"],
                "action_input": parsed["action_input"]
            })
            
            # 检查是否有 Final Answer
            if parsed["final_answer"]:
                if verbose:
                    print(f"\n✅ Final Answer: {parsed['final_answer']}")
                return parsed["final_answer"]
            
            # 执行工具
            if parsed["action"] and parsed["action"] in self.tools:
                tool = self.tools[parsed["action"]]
                action_input = parsed["action_input"] or {}
                
                if verbose:
                    print(f"\n🔧 Executing: {parsed['action']}")
                    print(f"   Input: {action_input}")
                
                # 执行工具
                observation = tool.run(**action_input)
                
                if verbose:
                    print(f"👁️ Observation: {observation}")
                
                # 更新历史
                self.history[-1]["observation"] = observation
                
                # 添加到对话历史
                messages.append({"role": "assistant", "content": response})
                messages.append({"role": "user", "content": f"Observation: {observation}"})
            
            elif parsed["action"]:
                # 工具不存在
                error_msg = f"Tool '{parsed['action']}' not found. Available tools: {list(self.tools.keys())}"
                messages.append({"role": "assistant", "content": response})
                messages.append({"role": "user", "content": f"Error: {error_msg}"})
                
                if verbose:
                    print(f"❌ {error_msg}")
            
            else:
                # 无法解析，让 LLM 重试
                messages.append({"role": "assistant", "content": response})
                messages.append({"role": "user", "content": "Please respond in the correct format with Thought, Action, Action Input, or Final Answer."})
        
        return "Max steps reached without finding an answer."

# 创建 Agent
agent = ReActAgent(llm, ALL_TOOLS)
print("\n✓ ReAct Agent 创建成功!")

---

## 3. 测试 Agent

In [ ]:
# 测试1: 数学计算
result = agent.run("What is the square root of 144 plus 25?")

In [ ]:
# 测试2: 日期时间
result = agent.run("What day of the week is it today?")

In [ ]:
# 测试3: 天气查询
result = agent.run("What's the weather like in Tokyo right now?")

In [ ]:
# 测试4: 网络搜索
result = agent.run("What is Python programming language?")

In [ ]:
# 测试5: 复合问题 (需要多步推理)
result = agent.run("If today is Monday and I need to wait 15 days, what day of the week will it be? Also calculate 15 * 7.")

---

## 4. 可视化 Agent 工作流程

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
plt.rcParams["font.sans-serif"] = ["Microsoft YaHei", "SimHei", "Noto Sans CJK SC", "Arial Unicode MS"]
plt.rcParams["axes.unicode_minus"] = False

def visualize_react_flow():
    """可视化 ReAct Agent 的工作流程"""
    fig, ax = plt.subplots(figsize=(16, 10))
    ax.set_xlim(0, 16)
    ax.set_ylim(0, 10)
    ax.axis('off')
    ax.set_title('ReAct Agent 工作流程 (真实 LLM 驱动)', fontsize=16, fontweight='bold', pad=20)
    
    # 颜色定义
    colors = {
        'input': '#E8F6F3',
        'thought': '#FCF3CF',
        'action': '#FADBD8',
        'observation': '#D6EAF8',
        'output': '#D5F5E3',
        'llm': '#E8DAEF'
    }
    
    # Step 1: 用户输入
    ax.add_patch(FancyBboxPatch((1, 8), 4, 1.5, boxstyle='round,pad=0.1',
                                facecolor=colors['input'], edgecolor='#1ABC9C', linewidth=2))
    ax.text(3, 8.75, 'User Question', fontsize=12, ha='center', fontweight='bold')
    ax.text(3, 8.25, '"What\'s the weather in Tokyo?"', fontsize=9, ha='center', style='italic')
    
    # 箭头
    ax.annotate('', xy=(3, 6.7), xytext=(3, 7.8),
               arrowprops=dict(arrowstyle='->', color='#666', lw=2))
    
    # LLM 处理
    ax.add_patch(FancyBboxPatch((1, 5.2), 4, 1.5, boxstyle='round,pad=0.1',
                                facecolor=colors['llm'], edgecolor='#9B59B6', linewidth=2))
    ax.text(3, 5.95, 'LLM (Real)', fontsize=12, ha='center', fontweight='bold')
    ax.text(3, 5.45, 'Ollama / OpenAI / HF', fontsize=9, ha='center')
    
    # Thought
    ax.add_patch(FancyBboxPatch((6, 7), 4, 1.2, boxstyle='round,pad=0.1',
                                facecolor=colors['thought'], edgecolor='#F39C12', linewidth=2))
    ax.text(8, 7.6, 'Thought', fontsize=11, ha='center', fontweight='bold')
    ax.text(8, 7.15, '"I need to use weather tool"', fontsize=8, ha='center', style='italic')
    
    ax.annotate('', xy=(5.8, 7.5), xytext=(5.2, 6.5),
               arrowprops=dict(arrowstyle='->', color='#666', lw=2))
    
    # Action
    ax.add_patch(FancyBboxPatch((6, 5.2), 4, 1.2, boxstyle='round,pad=0.1',
                                facecolor=colors['action'], edgecolor='#E74C3C', linewidth=2))
    ax.text(8, 5.8, 'Action: get_weather', fontsize=11, ha='center', fontweight='bold')
    ax.text(8, 5.35, '{"city": "Tokyo"}', fontsize=9, ha='center', family='monospace')
    
    ax.annotate('', xy=(8, 5.2), xytext=(8, 6.8),
               arrowprops=dict(arrowstyle='->', color='#666', lw=2))
    
    # Tool Execution
    ax.add_patch(FancyBboxPatch((11, 5.2), 4, 1.2, boxstyle='round,pad=0.1',
                                facecolor='#FDF2E9', edgecolor='#E67E22', linewidth=2))
    ax.text(13, 5.8, 'Tool Execution', fontsize=11, ha='center', fontweight='bold')
    ax.text(13, 5.35, 'Open-Meteo API Call', fontsize=9, ha='center')
    
    ax.annotate('', xy=(10.8, 5.8), xytext=(10.2, 5.8),
               arrowprops=dict(arrowstyle='->', color='#666', lw=2))
    
    # Observation
    ax.add_patch(FancyBboxPatch((11, 3), 4, 1.5, boxstyle='round,pad=0.1',
                                facecolor=colors['observation'], edgecolor='#3498DB', linewidth=2))
    ax.text(13, 3.85, 'Observation', fontsize=11, ha='center', fontweight='bold')
    ax.text(13, 3.25, 'Temperature: 22°C, Sunny', fontsize=9, ha='center')
    
    ax.annotate('', xy=(13, 3), xytext=(13, 5),
               arrowprops=dict(arrowstyle='->', color='#666', lw=2))
    
    # 循环箭头
    ax.annotate('', xy=(3, 6.7), xytext=(10.8, 3.5),
               arrowprops=dict(arrowstyle='->', color='#9B59B6', lw=2,
                              connectionstyle='arc3,rad=-0.3'))
    ax.text(7, 4.2, 'Loop until Final Answer', fontsize=9, ha='center', color='#9B59B6')
    
    # Final Answer
    ax.add_patch(FancyBboxPatch((5.5, 1), 5, 1.5, boxstyle='round,pad=0.1',
                                facecolor=colors['output'], edgecolor='#27AE60', linewidth=2))
    ax.text(8, 1.85, 'Final Answer', fontsize=12, ha='center', fontweight='bold')
    ax.text(8, 1.25, '"It\'s 22°C and sunny in Tokyo"', fontsize=10, ha='center')
    
    ax.annotate('', xy=(8, 2.5), xytext=(8, 4.8),
               arrowprops=dict(arrowstyle='->', color='#27AE60', lw=2))
    
    plt.tight_layout()
    plt.show()

visualize_react_flow()

print("\n ReAct 模式关键点:")
print("   1. 真实 LLM 进行推理 (不是硬编码)")
print("   2. 工具调用真实 API (天气、搜索等)")
print("   3. 迭代式思考直到找到答案")
print("   4. 可扩展的工具系统")

---

## 本节总结


1. **ReAct 模式**
   - Thought → Action → Observation 循环
   - 真实 LLM 驱动推理过程

2. **工具系统**
   - 统一的工具接口定义
   - 真实的 API 调用 (天气、搜索)
   - JSON Schema 参数规范

3. **LLM 后端**
   - 支持 OpenAI / Ollama / HuggingFace
   - 统一的接口，灵活切换

### 扩展练习

1. **添加新工具**: 实现一个翻译工具 (可用 LibreTranslate API)
2. **改进解析**: 使用结构化输出 (JSON mode) 提高可靠性
3. **添加记忆**: 让 Agent 记住之前的对话

In [ ]:
# 练习: 添加一个新工具
# 例如: 单位转换工具

def unit_converter(value: float, from_unit: str, to_unit: str) -> str:
    """
    单位转换工具
    支持: 温度 (C/F/K), 长度 (m/ft/km/mi), 重量 (kg/lb)
    """
    conversions = {
        # 温度
        ('c', 'f'): lambda x: x * 9/5 + 32,
        ('f', 'c'): lambda x: (x - 32) * 5/9,
        ('c', 'k'): lambda x: x + 273.15,
        ('k', 'c'): lambda x: x - 273.15,
        # 长度
        ('m', 'ft'): lambda x: x * 3.28084,
        ('ft', 'm'): lambda x: x / 3.28084,
        ('km', 'mi'): lambda x: x * 0.621371,
        ('mi', 'km'): lambda x: x / 0.621371,
        # 重量
        ('kg', 'lb'): lambda x: x * 2.20462,
        ('lb', 'kg'): lambda x: x / 2.20462,
    }
    
    key = (from_unit.lower(), to_unit.lower())
    if key in conversions:
        result = conversions[key](value)
        return f"{value} {from_unit} = {result:.2f} {to_unit}"
    else:
        return f"Conversion from {from_unit} to {to_unit} not supported"

# 测试
print(unit_converter(100, "c", "f"))
print(unit_converter(1, "km", "mi"))